In [1]:
"""
Baseline for CIL project on road segmentation.
This simple baseline consits of a CNN with two convolutional+pooling layers with a soft-max loss
"""

import gzip
import os
import sys
import urllib
import matplotlib.image as mpimg
from PIL import Image

import code

import tensorflow.python.platform

import numpy as np
import tensorflow as tf

import cil_helper as cil

In [2]:
from skimage.feature import greycomatrix, greycoprops

In [3]:
try:
    tf.app.flags.DEFINE_string('log_dir', '/tmp/tensorflow_lukas',
                           """Directory where to write event logs """
                           """and checkpoint.""")
except: 
    print(tf.app.flags.FLAGS.log_dir)
    
FLAGS = tf.app.flags.FLAGS

In [4]:
TRAINING_SIZE = 20
VALIDATION_SIZE = 5  # Size of the validation set.
TEST_SIZE = 50
SEED = 66478  # Set to None for random seed.
BATCH_SIZE = 32 # 64
NUM_EPOCHS = 12
RESTORE_MODEL = False # If True, restore existing model instead of training a new one
RECORDING_STEP = 100

# Set image patch size in pixels
# IMG_PATCH_SIZE should be a multiple of 4
# image size should be an integer multiple of this number!
IMG_PATCH_SIZE = 16

In [29]:
# returns a numpy array [Saturation, Lightness] of a pixel
def sat_light(rgb):
    Cmax = rgb.max()
    Cmin = rgb.min()
    d = Cmax - Cmin
    L = (Cmax + Cmin + 1) / 2 -.5
    S = d / (1 - (abs(2 * L - 1))) - .5
    if np.isnan(S):
        S = 0
    if np.isnan(L):
        L = 0
    return np.asarray([S, L], dtype=np.float32)

In [30]:
max_offset = 2 #feel free to change
LIGHTNESS_LEVELS = 8
LIGHTNESS_IDX = 4
#returns [contrast, correlation, energy, homogeneity]
def texture_features(image, y, x):
    ylow = max(0, y-max_offset)
    yhi = min(image.shape[0], y+max_offset+1)
    xlow = max(0, x-max_offset)
    xhi = min(image.shape[1], x+max_offset+1)
    #print([ylow,yhi,xlow,xhi])
    window = ((image[ylow:yhi, xlow:xhi, LIGHTNESS_IDX] + .5) * LIGHTNESS_LEVELS).astype(np.uint8)
    print(window)
    glcm = greycomatrix(window, [i for i in range(1,max_offset)], [0, np.pi/2], levels=LIGHTNESS_LEVELS, normed=True)
    contrast = greycoprops(glcm, 'contrast').mean()
    correlation = greycoprops(glcm, 'correlation').mean()
    energy = greycoprops(glcm, 'energy').mean()
    homogeneity = greycoprops(glcm, 'homogeneity').mean()
    #dissimilarity = greycoprops(glcm, 'dissimilarity').mean()
    return np.asarray([contrast, correlation*.5, energy-.5, homogeneity-.5], dtype=np.float32)

In [31]:
NUM_CHANNELS = 5 # RGB, Saturation, Lightness, Texture
def preprocess(img):
    img = np.asarray([[
                    np.append(px, sat_light(px))
                    for px in y] for y in img], dtype=np.float32)
    return img #np.asarray([[
                #    np.append(img[y,x], texture_features(img,y,x))
                 #   for x in range(len(img[y]))] for y in range(len(img))], dtype=np.float32)

In [32]:
# Extract label images
def extract_labels(filename, num_images, preprocess=False):
    """Extract the labels into a 1-hot matrix [image index, label index]."""
    gt_imgs = []
    for i in range(1, num_images+1):
        imageid = "satImage_%.3d" % i
        image_filename = filename + imageid + ".png"
        if os.path.isfile(image_filename):
            #print ('Loading ' + image_filename)
            img = mpimg.imread(image_filename)
            gt_imgs.append(img)
        else:
            print ('File ' + image_filename + ' does not exist')

    num_images = len(gt_imgs)
    gt_patches = [cil.img_crop(gt_imgs[i], IMG_PATCH_SIZE, IMG_PATCH_SIZE) for i in range(num_images)]
    data = np.asarray([gt_patches[i][j] for i in range(len(gt_patches)) for j in range(len(gt_patches[i]))])
    labels = np.asarray([value_to_class(np.mean(data[i])) for i in range(len(data))])

    # Convert to dense 1-hot representation.
    return labels.astype(np.float32)

In [33]:
def extract_data(filename, num_images, preprocess=None):
    """Extract the images into a 4D tensor [image index, y, x, channels].
    Values are rescaled from [0, 255] down to [-0.5, 0.5].
    """
    imgs = []
    for i in range(1, num_images+1):
        imageid = "satImage_%.3d" % i
        image_filename = filename + imageid + ".png"
        if os.path.isfile(image_filename):
            #print ('Loading ' + image_filename)
            img = mpimg.imread(image_filename)
            if preprocess:
                img = preprocess(img)
            imgs.append(img)
        else:
            print ('File ' + image_filename + ' does not exist')

    num_images = len(imgs)
    IMG_WIDTH = imgs[0].shape[0]
    IMG_HEIGHT = imgs[0].shape[1]
    N_PATCHES_PER_IMAGE = (IMG_WIDTH/IMG_PATCH_SIZE)*(IMG_HEIGHT/IMG_PATCH_SIZE)

    img_patches = [cil.img_crop(imgs[i], IMG_PATCH_SIZE, IMG_PATCH_SIZE) for i in range(num_images)]
    data = [img_patches[i][j] for i in range(len(img_patches)) for j in range(len(img_patches[i]))]

    return np.asarray(data)

In [34]:
# Assign a label to a patch v
def value_to_class(v):
    foreground_threshold = 0.5 # percentage of pixels > 1 required to assign a foreground label to a patch
    #ret#urn [1-v , v]
    df = np.sum(v)
    if df > foreground_threshold:
       return [0, 1]
    else:
       return [1, 0]

In [35]:
data_dir = 'training/'
train_data_filename = data_dir + 'images/'
train_labels_filename = data_dir + 'groundtruth/' 

# Extract it into numpy arrays.
train_data = extract_data(train_data_filename, TRAINING_SIZE, preprocess)
train_labels = extract_labels(train_labels_filename, TRAINING_SIZE)

num_epochs = NUM_EPOCHS

In [36]:
train_data.shape

(12500, 16, 16, 5)

In [37]:
c0 = 0
c1 = 0
for i in range(len(train_labels)):
    if train_labels[i][0] > 0.9:
        c0 = c0 + 1
    else:
        c1 = c1 + 1
print ('Number of data points per class: c0 = ' + str(c0) + ' c1 = ' + str(c1))

Number of data points per class: c0 = 10438 c1 = 2062


In [38]:
print ('Balancing training data...')
min_c = min(c0, c1)
idx0 = [i for i, j in enumerate(train_labels) if j[0] == 1]
idx1 = [i for i, j in enumerate(train_labels) if j[1] == 1]
new_indices = idx0[0:min_c] + idx1[0:min_c]
train_data = train_data[new_indices,:,:,:]
train_labels = train_labels[new_indices]

Balancing training data...


In [39]:
train_size = train_labels.shape[0]

In [40]:
# This is where training samples and labels are fed to the graph.
# These placeholder nodes will be fed a batch of training data at each
# training step using the {feed_dict} argument to the Run() call below.
train_data_node = tf.placeholder(
    tf.float32,
    shape=(BATCH_SIZE, IMG_PATCH_SIZE, IMG_PATCH_SIZE, NUM_CHANNELS),
    name='train_data')
train_labels_node = tf.placeholder(
    tf.float32,
    shape=(BATCH_SIZE, cil.NUM_LABELS),
    name='train_labels')
train_all_data_node = tf.constant(train_data)

In [41]:
# The variables below hold all the trainable weights. They are passed an
# initial value which will be assigned when when we call:
# {tf.initialize_all_variables().run()}
conv1_weights = cil.weight_variable([10, 10, NUM_CHANNELS, 40])
conv1_biases = cil.bias_variable([40])

conv2_weights = cil.weight_variable([8, 8, 40, 64])
conv2_biases = cil.bias_variable([64])

conv3_weights = cil.weight_variable([4, 4, 64, 200])
conv3_biases = cil.bias_variable([200])

conv4_weights = cil.weight_variable([4, 4, 200, 256])
conv4_biases = cil.bias_variable([256])

fc1_weights = tf.Variable(  # fully connected, depth 512.
    tf.truncated_normal([int(IMG_PATCH_SIZE / 4 * IMG_PATCH_SIZE / 4 * 64), 512],
                        stddev=0.1,
                        seed=SEED))
fc1_biases = tf.Variable(tf.constant(0.1, shape=[512]))
fc2_weights = tf.Variable(
    tf.truncated_normal([512, cil.NUM_LABELS],
                        stddev=0.1,
                        seed=SEED))
fc2_biases = tf.Variable(tf.constant(0.1, shape=[cil.NUM_LABELS]))
conv1_weights_hist = tf.histogram_summary("conv1_weights", conv1_weights)
conv2_weights_hist = tf.histogram_summary("conv2_weights", conv2_weights)
conv1_biases_hist = tf.histogram_summary("conv1_biases", conv1_biases)
conv2_biases_hist = tf.histogram_summary("conv2_biases", conv2_biases)
fc1_weights_hist = tf.histogram_summary("fc1_weights", fc1_weights)
fc1_biases_hist = tf.histogram_summary("fc1_biases", fc1_biases)
fc2_weights_hist = tf.histogram_summary("fc2_weights", fc2_weights)
fc2_biases_hist = tf.histogram_summary("fc2_biases", fc2_biases)

In [42]:
# We will replicate the model structure for the training subgraph, as well
# as the evaluation subgraphs, while sharing the trainable parameters.
def model(data, train=False):
    """The Model definition."""
    conv = cil.conv2d(data, conv1_weights)
    relu = tf.nn.relu(tf.nn.bias_add(conv, conv1_biases))
    pool = cil.max_pool_2x2(relu)

    conv2 = cil.conv2d(pool, conv2_weights)
    relu2 = tf.nn.relu(tf.nn.bias_add(conv2, conv2_biases))
    pool2 = cil.max_pool_2x2(relu2)
    
    conv3 = cil.conv2d(pool2, conv3_weights)
    relu3 = tf.nn.relu(tf.nn.bias_add(conv3, conv3_biases))
    
    conv4 = cil.conv2d(relu3, conv4_weights)
    relu4 = tf.nn.relu(tf.nn.bias_add(conv4, conv4_biases))
    
    pool4 = cil.max_pool_2x2(relu4)

    # Reshape the feature map cuboid into a 2D matrix to feed it to the
    # fully connected layers.
    pool_shape = pool4.get_shape().as_list()
    reshape = tf.reshape(
        pool4,
        [pool_shape[0], pool_shape[1] * pool_shape[2] * pool_shape[3]])
    # Fully connected layer. Note that the '+' operation automatically
    # broadcasts the biases.
    
    fc1 = tf.nn.relu(tf.matmul(reshape, fc1_weights) + fc1_biases)
    if train:
        fc1 = tf.nn.dropout(fc1, 0.6, seed=SEED)
    fc2 = tf.matmul(fc1, fc2_weights) + fc2_biases

    if train == True:
        summary_id = '_0'
        s_data = cil.get_image_summary(data)
        filter_summary0 = tf.image_summary('summary_data' + summary_id, s_data)
        s_conv = cil.get_image_summary(conv)
        filter_summary2 = tf.image_summary('summary_conv' + summary_id, s_conv)
        #s_pool = cil.get_image_summary(pool)
        #filter_summary3 = tf.image_summary('summary_pool' + summary_id, s_pool)
        s_conv2 = cil.get_image_summary(conv2)
        filter_summary4 = tf.image_summary('summary_conv2' + summary_id, s_conv2)
        s_pool2 = cil.get_image_summary(pool2)
        filter_summary5 = tf.image_summary('summary_pool2' + summary_id, s_pool2)

    return fc2

In [43]:
# Training computation: logits + cross-entropy loss.
logits = model(train_data_node, True) # BATCH_SIZE*NUM_LABELS
# print 'logits = ' + str(logits.get_shape()) + ' train_labels_node = ' + str(train_labels_node.get_shape())
#loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
#    logits, train_labels_node))
loss = tf.reduce_mean(tf.square(logits - train_labels_node))
loss_summary = tf.scalar_summary('loss', loss)

In [44]:
all_params_node = [conv1_weights, conv1_biases, conv2_weights, conv2_biases, fc1_weights, fc1_biases, fc2_weights, fc2_biases]
all_params_names = ['conv1_weights', 'conv1_biases', 'conv2_weights', 'conv2_biases', 'fc1_weights', 'fc1_biases', 'fc2_weights', 'fc2_biases']
all_grads_node = tf.gradients(loss, all_params_node)
all_grad_norms_node = []
for i in range(0, len(all_grads_node)):
    norm_grad_i = tf.global_norm([all_grads_node[i]])
    all_grad_norms_node.append(norm_grad_i)
    all_params_summary = tf.scalar_summary(all_params_names[i], norm_grad_i)

In [45]:
# L2 regularization for the fully connected parameters.
regularizers = (tf.nn.l2_loss(fc1_weights) + tf.nn.l2_loss(fc1_biases) +
                tf.nn.l2_loss(fc2_weights) + tf.nn.l2_loss(fc2_biases))
# Add the regularization term to the loss.
loss += 5e-4 * regularizers

In [46]:
# Optimizer: set up a variable that's incremented once per batch and
# controls the learning rate decay.
batch = tf.Variable(0)
# Decay once per epoch, using an exponential schedule starting at 0.01.
learning_rate = tf.train.exponential_decay(
    0.015,                # Base learning rate.
    batch * BATCH_SIZE,  # Current index into the dataset.
    train_size,          # Decay step.
    0.95,                # Decay rate.
    staircase=True)
learning_rate_summary = tf.scalar_summary('learning_rate', learning_rate)

In [47]:
# Use simple momentum for the optimization.
#optimizer = tf.train.MomentumOptimizer(learning_rate, 0.0).minimize(loss, global_step=batch)

# try adam
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step=batch)

In [48]:
# Predictions for the minibatch, validation set and test set.
train_prediction = tf.nn.softmax(logits)
# We'll compute them only once in a while by calling their {eval()} method.
train_all_prediction = tf.nn.softmax(model(train_all_data_node))

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

In [49]:
s = tf.InteractiveSession()

In [50]:
# Get prediction overlaid on the original image for given input file
def get_prediction_with_overlay(filename, image_idx):

    imageid = "satImage_%.3d" % image_idx
    image_filename = filename + imageid + ".png"
    img = mpimg.imread(image_filename)

    img_prediction = get_prediction(img)
    oimg = cil.make_img_overlay(img, img_prediction)

    return oimg

# Get a concatenation of the prediction and groundtruth for given input file
def get_prediction_with_groundtruth(filename, image_idx):

    imageid = "satImage_%.3d" % image_idx
    image_filename = filename + imageid + ".png"
    img = mpimg.imread(image_filename)

    img_prediction = get_prediction(img)
    cimg = cil.concatenate_images(img, img_prediction)

    return cimg

# Get prediction for given input image 
def get_prediction(img):
    data = np.asarray(cil.img_crop(img, IMG_PATCH_SIZE, IMG_PATCH_SIZE))
    data_node = tf.constant(data)
    output = tf.nn.softmax(model(data_node))
    output_prediction = s.run(output)
    img_prediction = cil.label_to_img(img.shape[0], img.shape[1], IMG_PATCH_SIZE, IMG_PATCH_SIZE, output_prediction)

    return img_prediction

In [51]:
if RESTORE_MODEL:
    # Restore variables from disk.
    saver.restore(FLAGS.train_dir + "/model.ckpt")
    print("Model restored.")

In [52]:
# Run all the initializers to prepare the trainable parameters.
tf.initialize_all_variables().run()

# Build the summary operation based on the TF collection of Summaries.
summary_op = tf.merge_summary([loss_summary, learning_rate_summary, all_params_summary, conv1_weights_hist, conv2_weights_hist])
summary_writer = tf.train.SummaryWriter(FLAGS.log_dir,
                                                graph=s.graph)
print ('Initialized!')

# Loop through training steps.
print ('Total number of iterations = ' + str(int(num_epochs * train_size / BATCH_SIZE)))

training_indices = range(train_size)

for iepoch in range(num_epochs):

    # Permute training indices
    perm_indices = np.random.permutation(training_indices)

    for step in range (int(train_size / BATCH_SIZE)):

        offset = (step * BATCH_SIZE) % (train_size - BATCH_SIZE)
        batch_indices = perm_indices[offset:(offset + BATCH_SIZE)]

        # Compute the offset of the current minibatch in the data.
        # Note that we could use better randomization across epochs.
        batch_data = train_data[batch_indices, :, :, :]
        batch_labels = train_labels[batch_indices]
        
        #print('batch_data.shape = ' + str(batch_data.shape))
        # This dictionary maps the batch data (as a numpy array) to the
        # node in the graph is should be fed to.
        feed_dict = {train_data_node: batch_data, train_labels_node: batch_labels}

        if step % RECORDING_STEP == 0:

            summary_str, _, l, lr, predictions = s.run(
                        [summary_op, optimizer, loss, learning_rate, train_prediction],
                        feed_dict=feed_dict)
            summary_writer.add_summary(summary_str, step)
            summary_writer.flush()
            
            l, lr, all_predictions = s.run(
                        [loss, learning_rate, train_all_prediction],
                        feed_dict=feed_dict)
            # print_predictions(predictions, batch_labels)

            #print('Epoch: ' + str(iepoch))
            #print('Step: ' + str(step))
            #print ('Minibatch loss: %.3f, learning rate: %.6f' % (l, lr))
            #print ('Minibatch error: %.1f%%' % cil.error_rate(predictions,
                                                                 #batch_labels))
            print('Step: ' + str(step + iepoch*int(train_size / BATCH_SIZE)) + ' error: %.1f%%' % cil.error_rate(all_predictions, train_labels))
            #print(batch_labels)
            sys.stdout.flush()
        else:
            # Run the graph and fetch some of the nodes.
            _, l, lr, predictions = s.run(
                        [optimizer, loss, learning_rate, train_prediction],
                        feed_dict=feed_dict)

# Save the variables to disk.
save_path = saver.save(s, FLAGS.train_dir + "/model.ckpt")
print("Model saved in file: %s" % save_path) 

Initialized!
Total number of iterations = 1546
Step: 0 error: 50.1%
Step: 100 error: 48.6%
Step: 128 error: 50.0%
Step: 228 error: 50.0%
Step: 256 error: 50.0%
Step: 356 error: 50.0%
Step: 384 error: 50.0%
Step: 484 error: 50.0%
Step: 512 error: 50.0%
Step: 612 error: 50.0%
Step: 640 error: 50.0%
Step: 740 error: 50.0%
Step: 768 error: 50.0%
Step: 868 error: 50.0%
Step: 896 error: 50.0%
Step: 996 error: 50.0%
Step: 1024 error: 50.0%
Step: 1124 error: 50.0%
Step: 1152 error: 50.0%
Step: 1252 error: 50.0%
Step: 1280 error: 50.0%
Step: 1380 error: 50.0%
Step: 1408 error: 50.0%
Step: 1508 error: 50.0%
Model saved in file: /tmp/tensorflow/model.ckpt


In [53]:
print ("Running prediction on training set")
prediction_training_dir = "predictions_training/"
if not os.path.isdir(prediction_training_dir):
    os.mkdir(prediction_training_dir)
for i in range(1, TRAINING_SIZE+1):
    pimg = get_prediction_with_groundtruth(train_data_filename, i)
    Image.fromarray(pimg).save(prediction_training_dir + "prediction_" + str(i) + ".png")
    oimg = get_prediction_with_overlay(train_data_filename, i)
    oimg.save(prediction_training_dir + "overlay_" + str(i) + ".png")  

Running prediction on training set


ValueError: Dimensions 3 and 5 are not compatible

In [ ]:
print ("Running prediction on test set")
prediction_test_dir = "predictions_test/"
test_dir = "test/"
if not os.path.isdir(prediction_test_dir):
    os.mkdir(prediction_test_dir)
for i in range(1, TEST_SIZE+1):
    imagename = "test_" + str(i)
    image_filename = test_dir + imagename + "/" + imagename + ".png"
    img = mpimg.imread(image_filename)
    pimg = cil.img_float_to_uint8(get_prediction(img))
    Image.fromarray(pimg).save(prediction_test_dir + "prediction_" + str(i) + ".png")

In [ ]:
import mask_to_submission as mts
mts.save('submission_lukas_10_1.csv')